<a href="https://colab.research.google.com/github/Bibi852/jhu-ml/blob/main/Module4Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install ISLP

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.0/522.0 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.2 MB/s eta 0:00:00
 

### Importing Packages and Stock Market Data

In [3]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
summarize)
from ISLP import confusion_table
from ISLP.models import contrast
from sklearn.discriminant_analysis import \
(LinearDiscriminantAnalysis as LDA ,
QuadraticDiscriminantAnalysis as QDA)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [4]:
Smarket = load_data('Smarket')

### Logistic Regression

In [5]:
allvars = Smarket.columns.drop(['Today', 'Direction', 'Year'])
design = MS(allvars)
X = design.fit_transform(Smarket)
y = Smarket.Direction == 'Up'
glm = sm.GLM(y, X, family = sm.families.Binomial())
results = glm.fit()
summarize(results)

,coef,std err,z,P>|z|
intercept,-0.1260,0.241,-0.523,0.601
Lag1,-0.0731,0.050,-1.457,0.145
Lag2,-0.0423,0.050,-0.845,0.398
Lag3,0.0111,0.050,0.222,0.824
Lag4,0.0094,0.050,0.187,0.851
Lag5,0.0103,0.050,0.208,0.835
Volume,0.1354,0.158,0.855,0.392


In [7]:
train = (Smarket.Year < 2005)
Smarket_train = Smarket.loc[train]
Smarket_test = Smarket.loc[~train]
Smarket_test.shape

(252, 9)

In [31]:
X_train, X_test = X.loc[train], X.loc[~train]
y_train, y_test = y.loc[train], y.loc[~train]
glm_train = sm.GLM(y_train, X_train, family = sm.families.Binomial())
results = glm_train.fit()
probs = results.predict(exog=X_test)

In [32]:
D = Smarket.Direction
L_train, L_test = D.loc[train], D.loc[~train]
labels = np.array(['Down']*252)
labels[probs>0.5] = 'Up'
confusion_table(labels, L_test)

Truth,Down,Up
Predicted,,
Down,35,35
Up,76,106


In [37]:
model = MS(['Lag1', 'Lag2']).fit(Smarket)
X = model.transform(Smarket)
X_train , X_test = X.loc[train], X.loc[~train]

### Linear Discriminant Analysis

In [38]:
lda = LDA(store_covariance = True)

In [39]:
# iterating over previous X train and test sets
X_train, X_test = [M.drop(columns = ['intercept']) for M in [X_train, X_test]]


In [40]:
lda.fit(X_train, L_train)

LinearDiscriminantAnalysis(store_covariance=True)

In [41]:
# extracting means of each class with the means_ attribute - average of each predictor in each class
lda.means_

array([[ 0.04279022,  0.03389409],
       [-0.03954635, -0.03132544]])

In [42]:
# prior probabilities - priors
print(f"Down prior probability: {lda.priors_[0]}\nUp prior probability: {lda.priors_[1]}")

Down prior probability: 0.49198396793587174
Up prior probability: 0.5080160320641283


In [43]:
# linear discriminant vectors
lda.scalings_

array([[-0.64201904],
       [-0.51352928]])

In [44]:
# vectors are multipliers of X = x - if scalings[0] times the first predictor (lag 1) - scalings[1] times second predictor (Lag2) is large - market increase,
# small - market decrease
lda_pred = lda.predict(X_test)

In [45]:
confusion_table(lda_pred, L_test)

Truth,Down,Up
Predicted,,
Down,35,35
Up,76,106


In [46]:
# classifies up when the posterior probability of up is above 0.5
lda_prob = lda.predict_proba(X_test)
np.all(
    np.where(lda_prob[:,1]>=0.5, 'Up', 'Down') == lda_pred
)



True

In [48]:
# first column denotes down class
lda_prob[:,0]

array([0.49017925, 0.4792185 , 0.46681848, 0.47400107, 0.49278766,
       0.49385615, 0.49510156, 0.4872861 , 0.49070135, 0.48440262,
       0.49069628, 0.51199885, 0.48951523, 0.47067612, 0.47445929,
       0.47995834, 0.49357753, 0.50308938, 0.49788061, 0.48863309,
       0.50065681, 0.51087353, 0.50399248, 0.49163351, 0.50417721,
       0.50267505, 0.49140429, 0.48059641, 0.48827181, 0.50621869,
       0.50059958, 0.49729649, 0.49585462, 0.48117774, 0.48414175,
       0.47263882, 0.48364175, 0.50910066, 0.51359414, 0.49338391,
       0.49268564, 0.4978472 , 0.49209142, 0.50563459, 0.50622877,
       0.48818939, 0.47252929, 0.48323391, 0.48350857, 0.49133344,
       0.48775664, 0.47243859, 0.48548774, 0.49329107, 0.48459731,
       0.47237179, 0.48161704, 0.49140673, 0.4942755 , 0.48412321,
       0.50260644, 0.50625572, 0.48218003, 0.48852631, 0.50118249,
       0.50005949, 0.50273766, 0.48700861, 0.48272133, 0.49965006,
       0.4818079 , 0.4651057 , 0.45778674, 0.47750037, 0.50342

### Quadratic Discriminant Analysis

In [49]:
qda = QDA(store_covariance = True)
qda.fit(X_train, L_train)

QuadraticDiscriminantAnalysis(store_covariance=True)

In [50]:
print(f"QDA Means: {qda.means_}\nQDA priors: {qda.priors_}")

QDA Means: [[ 0.04279022  0.03389409]
 [-0.03954635 -0.03132544]]
QDA priors: [0.49198397 0.50801603]


In [51]:
# predictions work the same way but coefficients do not since it is quadratic and not a linear equation
# whether covariance matrices are assumed to be equal determines complexity of discriminant function and thus whether linear or quadratic is used
# quadratic not as efficient for large number of p
qda_pred = qda.predict(X_test)
confusion_table(qda_pred, L_test)

Truth,Down,Up
Predicted,,
Down,30,20
Up,81,121


In [52]:
# higher accuracy - perhaps the relationship is non-linear
np.mean(qda_pred == L_test)

0.5992063492063492

### Naive Bayes

In [53]:
NB = GaussianNB()
NB.fit(X_train, L_train)
# Gaussian distribution is default, kernel density can also be used


GaussianNB()

In [54]:
print(f"Down prior prob: {NB.class_prior_[0]}\nUp prior prob: {NB.class_prior_[1]}")

Down prior prob: 0.49198396793587174
Up prior prob: 0.5080160320641283


In [55]:
# rows - classes, columns - features - theta[0, 0] - mean for feature lag 1 in the down class
NB.theta_

array([[ 0.04279022,  0.03389409],
       [-0.03954635, -0.03132544]])

In [58]:
nb_labels = NB.predict(X_test)
confusion_table(nb_labels, L_test)

Truth,Down,Up
Predicted,,
Down,29,20
Up,82,121


In [59]:
# slightly worse than QDA but better than LDA - LDA always prefered when nobs is small, p is large
# the performance of NB depends on whether the independence of predictors can be safely assumed
np.mean(nb_labels == L_test)

0.5952380952380952

### KNN

In [60]:
knn1 = KNeighborsClassifier(n_neighbors = 1)
knn1.fit(X_train, L_train)
knn1_pred = knn1.predict(X_test)
confusion_table(knn1_pred, L_test)

Truth,Down,Up
Predicted,,
Down,43,58
Up,68,83


In [61]:
# non-parametric model
# k = 1 may be too sensitive to data trends and thus have high variance
np.mean(knn1_pred == L_test)

0.5

In [63]:
knn3 = KNeighborsClassifier(n_neighbors = 3)
knn3.fit(X_train, L_train)
knn3_pred = knn3.predict(X_test)
confusion_table(knn3_pred, L_test)

Truth,Down,Up
Predicted,,
Down,48,55
Up,63,86


In [64]:
# there is always a cutoff for when increasing k results in no improvements
# according to textbook 3 is the max
np.mean(knn3_pred == L_test)

0.5317460317460317

#### KNN Caravan

In [68]:
Caravan = load_data("Caravan")
Purchase = Caravan.Purchase
Purchase.value_counts()

,count
Purchase,
No,5474
Yes,348


In [69]:
# scaling matters for KNN - variables on a larger scale will impact classification proportionally
# standardizing is needed
feature_df = Caravan.drop(columns = ['Purchase'])
scaler = StandardScaler(with_mean = True, with_std = True, copy = True)
scaler.fit(feature_df)
X_std = scaler.transform(feature_df)
# standard deviation of 1 and mean of 0
feature_std = pd.DataFrame(X_std, columns = feature_df.columns)
feature_std.std()

,0
MOSTYPE,1.000086
MAANTHUI,1.000086
MGEMOMV,1.000086
MGEMLEEF,1.000086
MOSHOOFD,1.000086
...,...
AZEILPL,1.000086
APLEZIER,1.000086
AFIETS,1.000086
AINBOED,1.000086


In [70]:
X_train, X_test, y_train, y_test = train_test_split(feature_std, Purchase, test_size = 1000, random_state = 0)

In [71]:
def knnfun(n):
  knn = KNeighborsClassifier(n_neighbors = n)
  knn_pred = knn.fit(X_train, y_train).predict(X_test)
  return knn_pred

In [73]:
knn1_pred = knnfun(1)

In [74]:
np.mean(y_test != knn1_pred), np.mean(y_test != 'No')

(0.111, 0.067)

In [75]:
# the null rate is 6% - the error rate if we always predicted no
confusion_table(knn1_pred, y_test)

Truth,No,Yes
Predicted,,
No,880,58
Yes,53,9


In [76]:
# when for those predicted to buy insurance - 14.5% are predicted correctly when compared to 6% if no is always chosen
9/62

0.14516129032258066

In [77]:
for K in range(1,6):
  knn_pred = knnfun(K)
  C = confusion_table(knn_pred, y_test)
  templ = ('K={0:d}: # predicted to rent: {1:>2},' + ' # who did rent {2:d}, accuracy {3:.1%}')
  pred = C.loc['Yes'].sum()
  did_rent = C.loc['Yes','Yes']
  print(templ.format(
    K,
    pred ,
    did_rent ,
    did_rent / pred))

K=1: # predicted to rent: 62, # who did rent 9, accuracy 14.5%
K=2: # predicted to rent:  6, # who did rent 1, accuracy 16.7%
K=3: # predicted to rent: 20, # who did rent 3, accuracy 15.0%
K=4: # predicted to rent:  4, # who did rent 0, accuracy 0.0%
K=5: # predicted to rent:  7, # who did rent 1, accuracy 14.3%


I didn't do the rest of the lab because it is not included in the homework.